# 🤖 OpenAI Agents SDK – Class 11 (1st November 2025)

### 📦 Install `openai-agents` package

In [ ]:
%pip install -qU openai-agents python-dotenv

### Make Jupyter Notebook capable of running asynchronous functions.

In [1]:
import nest_asyncio
nest_asyncio.apply()

### 🌐 Global Configurations for Connecting to Gemini

In [ ]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import set_default_openai_api, set_default_openai_client, set_tracing_disabled

# Setup for Api Keys
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Which LLM Service?
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Global Setup for Connecting to Gemini
set_tracing_disabled(disabled=True)
set_default_openai_client(client=external_client)
set_default_openai_api("chat_completions")


## Response Items

In [3]:
from agents import Agent, Runner

maths_agent = Agent(
    name="Maths Agent",
    model="gemini-2.5-flash",
)

agent = Agent(
    name="Traige Agent",
    model="gemini-2.5-flash",
    handoffs=[maths_agent],
)

result = Runner.run_sync(
    starting_agent=agent,
    input="What 2 + 2?"
)

display(result.to_input_list())

new_input = result.to_input_list() + [{"role": "user", "content": "Now add 15 in it."}]

result2 = Runner.run_sync(
    starting_agent=agent,
    input=new_input
)

display(result2.to_input_list())

[{'content': 'What 2 + 2?', 'role': 'user'},
 {'arguments': '{}',
  'call_id': 'function-call-947881898374134618',
  'name': 'transfer_to_maths_agent',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'function-call-947881898374134618',
  'output': '{"assistant": "Maths Agent"}',
  'type': 'function_call_output'},
 {'id': '__fake_id__',
  'content': [{'annotations': [], 'text': '2 + 2 = 4', 'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

[{'content': 'What 2 + 2?', 'role': 'user'},
 {'arguments': '{}',
  'call_id': 'function-call-947881898374134618',
  'name': 'transfer_to_maths_agent',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'function-call-947881898374134618',
  'output': '{"assistant": "Maths Agent"}',
  'type': 'function_call_output'},
 {'id': '__fake_id__',
  'content': [{'annotations': [], 'text': '2 + 2 = 4', 'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'role': 'user', 'content': 'Now add 15 in it.'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': '2 + 2 + 15 = 19',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

## Max turns

In [ ]:
from agents import Agent, Runner, function_tool

@function_tool
def add_two_numbers(a: int, b: int):
    return a + b


agent = Agent(
    name="Traige Agent",
    model="gemini-2.5-flash",
    tools=[add_two_numbers],
)

result = Runner.run_sync(
    starting_agent=agent,
    input="What 2 + 2?",
    max_turns=1
)

print(result.final_output)

MaxTurnsExceeded: Max turns (1) exceeded

## Model Settings
### Tool Choice

In [7]:
from agents import Agent, Runner, ModelSettings


@function_tool
def add_two_numbers(a: int, b: int):
    print(f"add_two_numbers function called with arguments: a={a}, b={b}")
    return a + b

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant. Answer user's query",
    model="gemini-2.5-flash",
    tools=[add_two_numbers],
    model_settings=ModelSettings(
        tool_choice="required",
    ),
)

result = Runner.run_sync(
    starting_agent=agent,
    input="Who is founder of pakistan",
)


print(result.final_output)
# See tracing

add_two_numbers function called with arguments: a=1, b=2
I can answer questions using numbers. I cannot answer general knowledge questions. Would you like me to help with a math question?


### Tool use behavior

In [ ]:
from agents import Agent, Runner


@function_tool
def add_two_numbers(a: int, b: int):
    return a + b - 10


agent = Agent(
    name="Traige Agent",
    model="gemini-2.5-flash",
    tools=[add_two_numbers],
    tool_use_behavior="stop_on_first_tool",
)

result = Runner.run_sync(
    starting_agent=agent,
    input="What 2 + 2?",
)

print(result.final_output)

-6


### Max tokens

In [20]:
from agents import Agent, Runner, ModelSettings

agent = Agent(
    name="Assistant",
    model="gemini-2.5-flash",
    model_settings=ModelSettings(max_tokens=100)
)

result = Runner.run_sync(
    starting_agent=agent,
    input="Write blog on AI in 500 words.",
)

print(result.final_output)

### Parallel Tool Calls

In [ ]:
from agents import Agent, Runner, ModelSettings
import time


@function_tool
def add_two_numbers(a: int, b: int):
    print("add_two_numbers function called")
    time.sleep(3) # Pause for 3 seconds
    print("add_two_numbers function end")

    return a + b

@function_tool
def multiply_two_numbers(a: int, b: int):
    print("multiply_two_numbers function called")
    time.sleep(2) # Pause for 2 seconds
    print("multiply_two_numbers function end")

    return a * b


agent = Agent(
    name="My Agent",
    model="gemini-2.5-flash",
    tools=[add_two_numbers, multiply_two_numbers],
    model_settings=ModelSettings(parallel_tool_calls=True),  # use gpt model here, or any model that supports parallel tool calls,
)

result = Runner.run_sync(
    starting_agent=agent,
    input="What 2 + 2 and 6 times 7?",
)

print(result.final_output)


## Streaming
### LLM Output Streaming

In [21]:
import asyncio
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent

agent = Agent(name="Joker", model="gemini-2.5-flash")

async def main():
    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")

    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

asyncio.run(main())


Here are 5 jokes for you!

1.  Why don't scientists trust atoms?
    *   Because they make up *everything*!

2.  What do you call a boomerang that won't come back?
    *   A stick.

3.  Knock, knock.
    *   Who's there?
    *   Interrupting cow.
    *   Interrupting co—
    *   MOOOOOOO!

4.  I told my wife she should embrace her mistakes.
    *   She gave me a hug.

5.  Why did the scarecrow win an award?
    *   Because he was outstanding in his field!

### Agent Loop Streaming

In [22]:
import asyncio
from agents import Agent, ItemHelpers, Runner

agent = Agent(name="Joker", model="gemini-2.5-flash")

async def main():
    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")

    print("=== Run starting ===")
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue

        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue

        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")


asyncio.run(main())


=== Run starting ===
Agent updated: Joker
-- Message output:
 Here are 5 jokes for you!

1.  **Why don't scientists trust atoms?**
    Because they make up everything!

2.  **What do you call a fake noodle?**
    An impasta!

3.  **Why did the scarecrow win an award?**
    Because he was outstanding in his field!

4.  **Knock, knock.**
    *Who's there?*
    Lettuce.
    *Lettuce who?*
    Lettuce in, it's cold out here!

5.  **What's an astronaut's favorite part of a computer?**
    The space bar!
=== Run complete ===


## Error in Tool (Agent Loop)

In [23]:
import asyncio
from agents import Agent, Runner, function_tool

@function_tool
def multiply(a: int, b: int):
    return a / 0

agent = Agent(name="Assistant", tools=[multiply], model="gemini-2.5-flash")
result = Runner.run_sync(agent, input="What is 7 by 7?")

print(result.final_output)

I'm sorry, I wasn't able to get the answer to your question. Please try again.


## Clone Agent

In [24]:
from agents import function_tool

@function_tool
def calculate_area(length: float, width: float) -> str:
    return f"Area = {length * width} square units"

@function_tool
def get_weather(city: str) -> str:
    return f"Weather in {city}: Sunny, 72°F"

# Base agent with one tool
base_agent = Agent(
    name="BaseAssistant",
    model="gemini-2.5-flash",
    tools=[calculate_area],
    instructions="You are a helpful assistant.",
)

# Clone with additional tool
weather_agent = base_agent.clone(
    name="WeatherAssistant",
    model="gemini-2.5-flash",
    tools=[calculate_area, get_weather],  # New tools list
    instructions="You are a weather and math assistant.",
)

# Clone with same tools
math_agent = base_agent.clone(
    name="MathAssistant",
    model="gemini-2.5-flash",
    instructions="You are a math specialist.",
)

print("Base Agent Tools: ", base_agent.tools)
print("Weather Agent Tools: ", [tool.name for tool in weather_agent.tools])
print("Math Agent Tools: ", [tool.name for tool in math_agent.tools])


Base Agent Tools:  [FunctionTool(name='calculate_area', description='', params_json_schema={'properties': {'length': {'title': 'Length', 'type': 'number'}, 'width': {'title': 'Width', 'type': 'number'}}, 'required': ['length', 'width'], 'title': 'calculate_area_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001DC6981A840>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)]
Weather Agent Tools:  ['calculate_area', 'get_weather']
Math Agent Tools:  ['calculate_area']


## Advance Tools

## Structure Output

In [25]:
from agents import Agent, Runner
from pydantic import BaseModel


class WeatherAnswer(BaseModel):
    location: str
    temperature_c: float
    summary: str

agent = Agent(
    name="Assistant",
    output_type=WeatherAnswer,
    model="gemini-2.5-flash",
)

result = Runner.run_sync(agent, "What's the weather in Karachi?")

# Perfect! Now you get structured data:
print(result.final_output)
print(result.final_output.location)      # "Karachi"
print(result.final_output.temperature_c) # 30.0
print(result.final_output.summary)       # "clear skies"

location='Karachi' temperature_c=30.0 summary='Partly cloudy'
Karachi
30.0
Partly cloudy


## Lifecyles
### Agent Hooks

In [26]:
from agents import AgentHooks

class TestAgHooks(AgentHooks):
    def __init__(self, ag_display_name):
        self.event_counter = 0
        self.ag_display_name = ag_display_name

    async def on_start(self, context, agent) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} started.")

    async def on_end(self, context, agent, output) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} ended. Output: {output}")

In [27]:
from typing import Any
from agents import AgentHooks, RunContextWrapper

class TestAgHooks(AgentHooks):
    def __init__(self, ag_display_name):
        self.event_counter = 0
        self.ag_display_name = ag_display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} started. Usage: {context.usage}")

    async def on_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} ended. Usage: {context.usage}, Output: {output}")

In [28]:
start_agent = Agent(
    name="Content Moderator Agent",
    instructions="You are content moderation agent. Watch social media content received and flag queries that need help or answer. We will answer anything about AI?",
    hooks=TestAgHooks(ag_display_name="content_moderator"),
    model="gemini-2.5-flash"
)

async def main():
  result = await Runner.run(
      start_agent,
      input=f"<tweet>Will Agentic AI Die at end of 2025?.</tweet>"
  )

  print(result.final_output)

asyncio.run(main())
print("--end--")

### content_moderator 1: Agent Content Moderator Agent started. Usage: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
### content_moderator 2: Agent Content Moderator Agent ended. Usage: Usage(requests=1, input_tokens=49, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=10, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=142), Output: This query asks about AI and needs an answer.
This query asks about AI and needs an answer.
--end--


## Runner Hooks

In [29]:
from agents import Agent, RunHooks, Runner

# Create a system-wide monitoring class
class SystemMonitor(RunHooks):
    def __init__(self):
        self.active_agents = []
        self.tool_usage = {}
        self.handoffs = 0

    async def on_agent_start(self, context, agent):
        self.active_agents.append(agent.name)
        print(f"🌅 SYSTEM: {agent.name} is now working")
        print(f"   Active agents so far: {self.active_agents}")

    async def on_llm_start(self, context, agent, system_prompt, input_items):
        print(f"📞 SYSTEM: {agent.name} is thinking...")

    async def on_llm_end(self, context, agent, response):
        print(f"🧠✨ SYSTEM: {agent.name} finished thinking")

    async def on_tool_start(self, context, agent, tool):
        tool_name = tool.name
        if tool_name not in self.tool_usage:
            self.tool_usage[tool_name] = 0
        self.tool_usage[tool_name] += 1
        print(f"🔨 SYSTEM: {tool_name} used {self.tool_usage[tool_name]} times")

    async def on_tool_end(self, context, agent, tool, result):
        print(f"✅🔨 SYSTEM: {agent.name} finished using {tool.name}")

    async def on_handoff(self, context, from_agent, to_agent):
        self.handoffs += 1
        print(f"🏃‍♂️➡️🏃‍♀️ HANDOFF #{self.handoffs}: {from_agent.name} → {to_agent.name}")

    async def on_agent_end(self, context, agent, output):
        print(f"✅ SYSTEM: {agent.name} completed their work")
        print(f"📊 STATS: {len(self.active_agents)} agents used, {self.handoffs} handoffs")


# Create your agents
tech_support = Agent(name="TechnicalSupport", model="gemini-2.5-flash")
billing_manager = Agent(name="BillingManager", model="gemini-2.5-flash")
customer_service = Agent(name="CustomerService", model="gemini-2.5-flash", handoffs=[tech_support, billing_manager])

# Create the system monitor
system_monitor = SystemMonitor()

# Run with system-wide monitoring
result = Runner.run_sync(
    starting_agent=customer_service,
    input="I need help with my account",
    hooks=system_monitor,  # This monitors EVERYTHING
)

print(result.final_output)


🌅 SYSTEM: CustomerService is now working
   Active agents so far: ['CustomerService']
📞 SYSTEM: CustomerService is thinking...
🧠✨ SYSTEM: CustomerService finished thinking
✅ SYSTEM: CustomerService completed their work
📊 STATS: 1 agents used, 0 handoffs
Could you please tell me more about the help you need with your account? For example, are you having a technical issue, or do you have a question about billing?
